In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException   
import time
from time import sleep
import csv
import pandas
import numpy as np
import unicodedata

In [2]:
# Función descargaPaginaWeb que permite recuperar la información correspondiente a la respuesta de la petición. 
# url: corresponde con la url del sitio donde se hará el web scraping
def descargaPaginaWeb(url):
    perfil = webdriver.FirefoxProfile()
    opciones = webdriver.FirefoxOptions()
    # la ventana se hace visible para ver cómo actúa el scraper
    opciones.headless = False
    # se especifica el user-agent
    perfil.set_preference("general.useragent.override", "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:93.0) Gecko/20100101 Firefox/93.0")
    # creamos el objeto driver con el perfil y opciones indicadas anteriormente y accede a la página web
    driver = webdriver.Firefox(firefox_profile=perfil,options=opciones)
    driver.get(url)
    # se maximiza la ventana que se ha hecho visible previamente
    driver.maximize_window()
    return driver

# Función seleccion_extraccion_datos que extrae de la página web del IGN la información relacionada con los sismos
# en España. La función retorna un dataframe con la información y el nombre de los campos.
# driver: driver utilizado para acceder a los datos de la página web
# fechaInicial: fecha inicial de búsqueda. Un valor vacío indica que se coge el valor por defecto.
# fechaFinal: fecha final de búsqueda. Un valor vacío indica que se coge el valor por defecto.
# intMin: intensidad mínima del sismo. Un valor vacío indica que no se filtra por intensidad.
# intMax: intensidad máxima del sismo. Un valor vacío indica que no se filtra por intensidad.
# magMin: magnitud mínima del sismo. Un valor vacío indica que no se filtra por magnitud.
# magMax: magnitud máxima del sismo. Un valor vacío indica que no se filtra por magnitud.
# profMin: profundidad mínima del sismo. Un valor vacío indica que no se filtra por profundidad.
# profMax: profundidad máxima del sismo. Un valor vacío indica que no se filtra por profundidad.
def seleccion_extraccion_datos(driver, fechaInicial, fechaFinal, intMin, intMax, magMin, magMax, profMin, profMax):
    seleccion_datos(driver, fechaInicial, fechaFinal, intMin, intMax, magMin, magMax, profMin, profMax)
    df = extraer_datos_paginas(driver)
    return df
# Función seleccion_datos que gestiona el filtrado de datos usando el formulario de la web
# driver: driver utilizado para acceder a los datos de la página web
# fechaInicial: fecha inicial de búsqueda. Un valor vacío indica que se coge el valor por defecto.
# fechaFinal: fecha final de búsqueda. Un valor vacío indica que se coge el valor por defecto.
# intMin: intensidad mínima del sismo. Un valor vacío indica que no se filtra por intensidad.
# intMax: intensidad máxima del sismo. Un valor vacío indica que no se filtra por intensidad.
# magMin: magnitud mínima del sismo. Un valor vacío indica que no se filtra por magnitud.
# magMax: magnitud máxima del sismo. Un valor vacío indica que no se filtra por magnitud.
# profMin: profundidad mínima del sismo. Un valor vacío indica que no se filtra por profundidad.
# profMax: profundidad máxima del sismo. Un valor vacío indica que no se filtra por profundidad.
def seleccion_datos(driver, fechaInicial, fechaFinal, intMin, intMax, magMin, magMax, profMin, profMax):
    if (fechaInicial != ""):
        fecha_inicial = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, 
            "//*[@id='_IGNSISCatalogoTerremotos_WAR_IGNSISCatalogoTerremotosportlet_startDate']")))                                                                                                                                                                                                                             
        fecha_inicial.clear()
        fecha_inicial.send_keys(fechaInicial) 
    if (fechaFinal != ""):
        fecha_final = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, 
            "//*[@id='_IGNSISCatalogoTerremotos_WAR_IGNSISCatalogoTerremotosportlet_endDate']")))       
        fecha_final.clear()
        fecha_final.send_keys(fechaFinal) 
    condicion_final = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, 
            "//*[@id='_IGNSISCatalogoTerremotos_WAR_IGNSISCatalogoTerremotosportlet_cond']"))) 
    condicion_final.send_keys('O')
    if (intMin != "" and intMax != ""):
        int_final = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, 
            "//*[@id='_IGNSISCatalogoTerremotos_WAR_IGNSISCatalogoTerremotosportlet_selIntensidad']"))) 
        int_final.send_keys('Sí')
        int_min_final = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, 
            "//*[@id='_IGNSISCatalogoTerremotos_WAR_IGNSISCatalogoTerremotosportlet_intMin']"))) 
        int_min_final.send_keys(intMin)
        int_max_final = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, 
            "//*[@id='_IGNSISCatalogoTerremotos_WAR_IGNSISCatalogoTerremotosportlet_intMax']"))) 
        int_max_final.send_keys(intMax)
    else:
        int_final.send_keys('No')
    if (magMin != "" and magMax != ""):
        mag_final = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, 
            "//*[@id='_IGNSISCatalogoTerremotos_WAR_IGNSISCatalogoTerremotosportlet_selMagnitud']"))) 
        mag_final.send_keys('Sí')
        mag_min_final = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, 
            "//*[@id='_IGNSISCatalogoTerremotos_WAR_IGNSISCatalogoTerremotosportlet_magMin']"))) 
        mag_min_final.send_keys(magMin)
        mag_max_final = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, 
            "//*[@id='_IGNSISCatalogoTerremotos_WAR_IGNSISCatalogoTerremotosportlet_magMax']"))) 
        mag_max_final.send_keys(magMax)
    else:
        mag_final.send_keys('No')    
    if (profMin != "" and profMax != ""):
        prof_final = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, 
            "//*[@id='_IGNSISCatalogoTerremotos_WAR_IGNSISCatalogoTerremotosportlet_selProf']"))) 
        prof_final.send_keys('Sí')
        prof_min_final = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, 
            "//*[@id='_IGNSISCatalogoTerremotos_WAR_IGNSISCatalogoTerremotosportlet_profMin']"))) 
        prof_min_final.send_keys(profMin)
        prof_max_final = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, 
            "//*[@id='_IGNSISCatalogoTerremotos_WAR_IGNSISCatalogoTerremotosportlet_profMax']"))) 
        prof_max_final.send_keys(profMax)
    else:
        prof_final.send_keys('No') 
    
    driver.execute_script("arguments[0].click();", driver.find_element_by_xpath("//*[@id='enviar']"));

    
# Función extraccion_datos que extrae de una página web la información requerida. La función retorna un dataframe con la 
# información de todos los sismos producidos y el nombre de los campos.
# driver: driver utilizado para acceder a los datos de la página web
def extraccion_datos(driver):
   # Número de eventos registrados en la página
    filas = len(driver.find_elements_by_xpath("/html/body/div[2]/div[3]/div/div/div/div/div/div/div/div/table/tbody/tr")) + 1

    # Número de campos de cada evento
    campos = len(driver.find_elements_by_xpath("/html/body/div[2]/div[3]/div/div/div/div/div/div/div/div/table/tbody/tr[1]/th"))
 
    # Se obtiene el nombre de los campos
    nombre_campos = []
    for c in range(1, campos):
        # obtaining the text from each column of the table
        valor = driver.find_element_by_xpath("/html/body/div[2]/div[3]/div/div/div/div/div/div/div/div/table/tbody/tr[1]/th[" + str(c) + "]").text
        valor = arreglarCampo(valor).strip().replace('\n','')
        nombre_campos.append(valor)
    
    # Se obtienen los valores de cada fila de eventos
    sismos = []
    for f in range(2, filas):
        evento = []
        for c in range(1, campos):
            valor = driver.find_element_by_xpath("/html/body/div[2]/div[3]/div/div/div/div/div/div/div/div/table/tbody/tr[" + str(f) + "]/td[" + str(c) + "]").text
            evento.append(valor)
        sismos.append(evento)
    
    # Se crea un dataframe con la información
    sismos_df = pandas.DataFrame(sismos, columns = nombre_campos)
    return sismos_df

# Función que elimina los acentos y la ñ de los campos de la tabla
def arreglarCampo(campo):
    s = ''.join(c for c in unicodedata.normalize('NFD', campo) if unicodedata.category(c) != 'Mn')
    return s

# Función extraer_datos_paginas que navega y extrae la información requerida de todas las paginas. La función retorna un dataframe con la 
# información de todos los sismos producidos y el nombre de los campos.
# driver: driver utilizado para acceder a los datos de la página web
def extraer_datos_paginas(driver):
    next_page = True
    sismos = None
    while next_page:
        driver.implicitly_wait(3) 
        if sismos is None:
            sismos = extraccion_datos(driver)
        else:
            sismos.append(extraccion_datos(driver))
        try:
            next_button = driver.find_element_by_xpath("//a[@title='Siguiente']")
            wait = WebDriverWait(driver, 10)
            element = wait.until(EC.element_to_be_clickable(next_button))
            next_button.click()
        except NoSuchElementException:
            next_page = False
    return sismos
    
    
# Función crearArchivoCSVDesdeDF que crea un archivo csv a partir de una lista de diccionarios.
# dataFrame: data frame con la información
# nombreArchivo: nombre del archivo csv
def crearArchivoCSVDesdeDF(dataFrame, nombreArchivo):
    dataFrame.to_csv(nombreArchivo, index = False)


In [3]:
# Función principal
def main():
    fecha_inicio = "28/10/2021"
    fecha_fin = "29/10/2021"
    driver = descargaPaginaWeb('https://www.ign.es/web/ign/portal/sis-catalogo-terremotos')
    seleccion_sismos_df = seleccion_extraccion_datos(driver,fecha_inicio ,fecha_fin , "I", "IV", "2", "5", "0", "30")
    crearArchivoCSVDesdeDF(seleccion_sismos_df, 'sismos_'+fecha_inicio+'_'+fecha_fin+'.csv')


In [4]:
main()

C:\Users\Eva\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  after removing the cwd from sys.path.
C:\Users\Eva\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Eva\Anaconda3\lib\site-packages\ipykernel_launcher.py:93: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
C:\Users\Eva\Anaconda3\lib\site-packages\ipykernel_launcher.py:101: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
C:\Users\Eva\Anaconda3\lib\site-packages\ipykernel_launcher.py:104: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
C:\Users\Eva\Anaconda3\lib\site-packages\ipykernel_launcher.py:112: DeprecationWarning: find_el

StaleElementReferenceException: Message: The element reference of <td> is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed
Stacktrace:
WebDriverError@chrome://marionette/content/error.js:181:5
StaleElementReferenceError@chrome://marionette/content/error.js:442:5
element.resolveElement@chrome://marionette/content/element.js:687:11
evaluate.fromJSON@chrome://marionette/content/evaluate.js:232:26
evaluate.fromJSON@chrome://marionette/content/evaluate.js:240:29
receiveMessage@chrome://marionette/content/actors/MarionetteCommandsChild.jsm:74:29
